__качаем объявления о продаже недвижимости с avito.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
import logging
import sys 

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [2]:
from time import sleep
from tqdm.auto import tqdm
import pandas as pd
from datetime import datetime 

pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

In [3]:
# !mkdir -p data/2024/raw/flat-arenda/avito

In [4]:
source = [
    # {
    #         'place': 'sevastopol',
    #     'subfolder': 'flat-arenda',
    #           'url': 'https://www.avito.ru/sevastopol/kvartiry/sdam/na_dlitelnyy_srok'
    # },
    {
            'place': 'sevastopol',
        'subfolder': 'flat',
              'url': 'https://www.avito.ru/sevastopol/kvartiry/prodam',
    },{
            'place': 'sevastopol',
        'subfolder': 'land',
              'url': 'https://www.avito.ru/sevastopol/zemelnye_uchastki/prodam',
    },{
            'place': 'sevastopol',
        'subfolder': 'house',
              'url': 'https://www.avito.ru/sevastopol/doma_dachi_kottedzhi/prodam',
    },
    {
            'place': 'sudak',
        'subfolder': 'flat',
              'url': 'https://www.avito.ru/sudak/kvartiry/prodam',
    },
    {
            'place': 'sudak',
        'subfolder': 'land',
              'url': 'https://www.avito.ru/sudak/zemelnye_uchastki/prodam',
    },{
            'place': 'sudak',
        'subfolder': 'house',
              'url': 'https://www.avito.ru/sudak/doma_dachi_kottedzhi/prodam',
    },
#     {
#             'place': 'bahchisaray',
#         'subfolder': 'flat',
#               'url': 'https://www.avito.ru/bahchisaray/kvartiry/prodam',
#     }, 
    {
            'place': 'bahchisaray',
        'subfolder': 'land',
              'url': 'https://www.avito.ru/bahchisaray/zemelnye_uchastki/prodam',
    },{
            'place': 'bahchisaray',
        'subfolder': 'house',
              'url': 'https://www.avito.ru/bahchisaray/doma_dachi_kottedzhi/prodam',
    },

]

---

In [5]:
# from selenium import webdriver
# from lib.avito import AvitoPageParser

# browser = webdriver.Firefox()
# browser.get(source[0]['url'])
# data = AvitoPageParser().transform(browser)
# # browser.close()

In [6]:
# browser.close()

In [7]:
# data

---

In [8]:
from selenium import webdriver
# from selenium.webdriver.common.by import By
from lib.avito import AvitoParser

In [9]:
dest_folder = 'data/2024/raw'

In [10]:
%%time 


options = webdriver.FirefoxOptions()
#options.headless = True
browser = webdriver.Firefox(options=options)
logging.info('open virtual browser')

for s in source:

    url = s['url']
    place = s['place']
    subfolder = s['subfolder'] 
    
    display(url)
    browser.get(url)
    sleep(3)
    
    data = AvitoParser().transform(browser)
    assert len(data)>0
    
    df = pd.DataFrame(data)
    df['ts'] = datetime.now()
    df['place'] = place

    display( len(df) )
    t_now = datetime.now().strftime('%Y-%m-%d_%H-%M')
    fname = f'{dest_folder}/{subfolder}/avito/avito_{t_now}_raw_{place}.pkl'
    df.to_pickle(fname)
    display( fname )

browser.close()

[INFO    ] 2024-10-15 11:46:32 | open virtual browser


'https://www.avito.ru/sevastopol/kvartiry/sdam/na_dlitelnyy_srok'

[INFO    ] 2024-10-15 11:46:46 | page 1: 50 items
[INFO    ] 2024-10-15 11:46:56 | page 2: 50 items
[INFO    ] 2024-10-15 11:47:05 | page 3: 50 items
[INFO    ] 2024-10-15 11:47:14 | page 4: 50 items
[INFO    ] 2024-10-15 11:47:23 | page 5: 50 items
[INFO    ] 2024-10-15 11:47:34 | page 6: 50 items
[INFO    ] 2024-10-15 11:47:43 | page 7: 50 items
[INFO    ] 2024-10-15 11:47:52 | page 8: 50 items
[INFO    ] 2024-10-15 11:48:01 | page 9: 50 items
[INFO    ] 2024-10-15 11:48:09 | page 10: 50 items
[INFO    ] 2024-10-15 11:48:18 | page 11: 50 items
[INFO    ] 2024-10-15 11:48:27 | page 12: 50 items
[INFO    ] 2024-10-15 11:48:37 | page 13: 50 items
[INFO    ] 2024-10-15 11:48:47 | page 14: 50 items
[INFO    ] 2024-10-15 11:48:57 | page 15: 50 items
[INFO    ] 2024-10-15 11:49:06 | page 16: 50 items
[INFO    ] 2024-10-15 11:49:16 | page 17: 50 items
[INFO    ] 2024-10-15 11:49:25 | page 18: 50 items
[INFO    ] 2024-10-15 11:49:32 | page 19: 37 items


937

'data/2024/raw/flat-arenda/avito/avito_2024-10-15_11-49_raw_sevastopol.pkl'

CPU times: user 21.8 s, sys: 941 ms, total: 22.8 s
Wall time: 3min 4s


In [11]:
# browser.close()

In [12]:
!date

Вт 15 окт 2024 11:49:35 MSK
